# Keşifsel Veri Analizi (EDA)

Bu notebook `bilet_satislar.csv` veri seti için temel keşifsel analizleri içerir: veri yükleme, eksik değer kontrolü, temel istatistikler ve birkaç görselleştirme. Türkçe etiketler kullanılmıştır.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline

# Veri yükle
df = pd.read_csv('bilet_satislar.csv', encoding='utf-8')
print('Yüklendi, boyut:', df.shape)

In [ ]:
# İlk satırlar ve bilgi
display(df.head())
display(df.info())

In [ ]:
# Eksik değerler
display(df.isnull().sum())

In [ ]:
# Yeni sütun: Toplam Gelir = Birim_Fiyat * Satilan_Adet
df['Birim_Fiyat'] = pd.to_numeric(df['Birim_Fiyat'], errors='coerce')
df['Satilan_Adet'] = pd.to_numeric(df['Satilan_Adet'], errors='coerce')
df['Toplam_Gelir'] = df['Birim_Fiyat'] * df['Satilan_Adet']
display(df.head())

In [ ]:
# Türkçe etiketli özet istatistikler
desc = df.describe(include='all')
desc = desc.rename(index={
    'count': 'Adet',
    'unique': 'Benzersiz',
    'top': 'En Sık',
    'freq': 'Sıklık',
    'mean': 'Ortalama',
    'std': 'Std',
    'min': 'Min',
    '50%': 'Medyan',
    'max': 'Maks'
})
display(desc)

In [ ]:
# Kategoriye göre toplam gelir ve adet
cat = df.groupby('Kategori').agg({'Toplam_Gelir':'sum','Satilan_Adet':'sum'}).sort_values('Toplam_Gelir', ascending=False)
display(cat)

In [ ]:
# Görselleştirme: Kategoriye göre toplam gelir
plt.figure(figsize=(8,5))
sns.barplot(x=cat.index, y=cat['Toplam_Gelir'].values)
plt.title('Kategoriye Göre Toplam Gelir')
plt.ylabel('Toplam Gelir')
plt.xlabel('Kategori')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Tarih sütununu datetime yap ve aylık satış trendi (Türkçe ay adları)
df['Tarih'] = pd.to_datetime(df['Tarih'], errors='coerce')
monthly = df.set_index('Tarih').resample('M').agg({'Toplam_Gelir':'sum','Satilan_Adet':'sum'})
months_tr = ['Ocak','Şubat','Mart','Nisan','Mayıs','Haziran','Temmuz','Ağustos','Eylül','Ekim','Kasım','Aralık']
labels = [f"{idx.year} {months_tr[idx.month-1]}" for idx in monthly.index]
plt.figure(figsize=(10,5))
plt.plot(labels, monthly['Toplam_Gelir'], marker='o')
plt.title('Aylık Toplam Gelir')
plt.ylabel('Toplam Gelir')
plt.xlabel('Ay')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# En çok satan 10 etkinlik
top = df.groupby('Urun_Adi').agg({'Satilan_Adet':'sum','Toplam_Gelir':'sum'}).sort_values('Satilan_Adet',ascending=False).head(10)
display(top)

## Kısa Notlar
- Eksik değerleri `Birim_Fiyat` veya `Satilan_Adet` için doldurmaya karar verebilirsin (median veya 0).
- Model eğitimi öncesi `Toplam_Gelir` veya tarih parçalarını özellik olarak eklemek faydalı olacaktır.